In [2]:
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
def openpickle(filename):
    with open(filename, "rb") as readfile:
        loaded = pickle.load(readfile)
    return loaded

In [5]:
train_pricing_decisions = pd.read_csv('train_prices_decisions.csv')
train_covariate = openpickle('train_covariate')
train_noisy_embedding = openpickle('train_noisy_embedding')
test_covariate = openpickle('test_covariate')
test_noisy_embedding = openpickle('test_noisy_embedding')
item0_embedding = openpickle('item0embedding')
item1_embedding = openpickle('item1embedding')

In [6]:
#Model to predict unknown user vectors
knn = KNeighborsRegressor(n_neighbors=78, metric='euclidean')
knn.fit(train_covariate[train_covariate.index.isin(train_noisy_embedding.index)].values, train_noisy_embedding.values)

picklefile = open('agents/thethreemusketeers_files/user_model', 'wb')
pickle.dump(knn, picklefile)

In [8]:
#Model to predict demand
unknown_user_vectors = knn.predict(train_covariate[~train_covariate.index.isin(train_noisy_embedding.index)].values)
predicted_user_vectors = pd.DataFrame(unknown_user_vectors, columns=range(10),
                                      index=train_covariate[~train_covariate.index.isin(train_noisy_embedding.index)].index)
combined_user_vectors = pd.concat([train_noisy_embedding, predicted_user_vectors])
combined_user_vectors.sort_index(inplace=True)
item_mat = np.array([item0_embedding, item1_embedding])
user_ratings = np.matmul(combined_user_vectors, item_mat.T)
user_ratings.rename(columns={0: "rating_0", 1: "rating_1"}, inplace=True)
df_train = train_pricing_decisions.join(train_covariate.reset_index(drop=True))
df_train.index = df_train['user_index']
df_train.drop(columns=['user_index'], inplace=True)
df_train.index.name = None
df_train_with_ratings = df_train.join(user_ratings)
clf = LogisticRegression().fit(df_train_with_ratings.drop(columns=['item_bought']).values,
                               df_train_with_ratings['item_bought'].values)

picklefile = open('agents/thethreemusketeers_files/demand_model', 'wb')
pickle.dump(clf, picklefile)

In [9]:
print(clf.score(df_train_with_ratings.drop(columns=['item_bought']).values,
                df_train_with_ratings['item_bought'].values))

0.9314285714285714
